# 1.8. Adding more metrics to gain a better understanding

## Imports

In [ ]:
import numpy as np # conda install numpy
import tensorflow as tf 
import matplotlib.pyplot as plt # conda install matplotlib
import pandas as pd # conda install pandas
import warnings
import seaborn as sns # conda install seaborn

warnings.filterwarnings('ignore')

print(tf.__version__)

## Load data and take a look at it

In [ ]:
data = pd.read_csv('airbnb new york.csv').sample(frac=1)
data.describe()

## Data preprocessing

In [ ]:
features = data[['neighbourhood_group', 'room_type', 'minimum_nights', 'number_of_reviews', 
                 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']]
#print(features.isna().sum())
features['reviews_per_month'] = features['reviews_per_month'].fillna(0)
#print(features.isna().sum())

onehot_neighborhood_group = pd.get_dummies(features['neighbourhood_group'])
onehot_room_type = pd.get_dummies(features['room_type'])
#print(onehot_room_type)

features = features.drop(columns=['neighbourhood_group', 'room_type'])
features = pd.concat([features, onehot_neighborhood_group, onehot_room_type], axis=1)
#print(features.head())

targets = data['price']

train_size = int(0.7*len(data))
X_train, X_test = features.values[:train_size, :], features.values[train_size:, :]
y_train, y_test = targets.values[:train_size], targets.values[train_size:]

print(len(X_train[0]))

## Data visualization and analysis

In [ ]:
sns.pairplot(data)

In [ ]:
corr = data.corr()
cmap = sns.diverging_palette(250, 10, as_cmap=True)
plt.figure(figsize=(8, 8))
sns.heatmap(corr, square=True, cmap=cmap, annot=True)

## The Tensorflow 2 Machine Learning Approaches

### Linear Regression

In [ ]:
class LinearModel:
    def __init__(self):
        # y_pred = W*X + b
        self.initializer = tf.keras.initializers.GlorotUniform()
    
    def loss(self, y, y_pred):
        return tf.reduce_mean(tf.abs(y - y_pred))
    
    def train(self, X, y, lr=0.00001, epochs=20, verbose=True):
            
        X = np.asarray(X, dtype=np.float32)
        y = np.asarray(y, dtype=np.float32).reshape((-1, 1)) # [1,2,3,4] -> [[1],[2],[3],[4]]
        
        self.W = tf.Variable(
            initial_value=self.initializer(shape=(len(X[0]), 1), dtype='float32'))
        self.b = tf.Variable(
            initial_value=self.initializer(shape=(1,), dtype='float32'))

        def train_step():
            with tf.GradientTape() as t:
                current_loss = self.loss(y, self.predict(X))

            dW, db = t.gradient(current_loss, [self.W, self.b])
            self.W.assign_sub(lr * dW) # W -= lr * dW
            self.b.assign_sub(lr * db)
            
            return current_loss
        
        for epoch in range(epochs):
            current_loss = train_step()
            if verbose:
                print(f'Epoch {epoch}: Loss: {current_loss.numpy()}') # <3 eager execution
                
    def predict(self, X):
        # [a, b] x [b, c]
        # X -> [n_instances, n_features], W -> [n_features, 1]
        return tf.matmul(X, self.W) + self.b

In [ ]:
model = LinearModel()
model.train(X_train, y_train, epochs=100)

## Multi Layer Perceptron (Artificial Neural Network)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'), # 0 or x: _____/
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

In [16]:
def R_squared(y_true, y_pred):
    residual = tf.reduce_sum(tf.square(y_true - y_pred))
    total = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    r2 = 1.0 - residual / total
    return r2

adam_optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.MAE
model.compile(
    optimizer=adam_optimizer,
    loss=loss_fn,
    metrics=[
        tf.keras.metrics.MAE,
        tf.keras.metrics.MSE,
        R_squared, # -1 and 1, < 0 => useless, 0 and 1 => better close to 1
    ]
)

In [17]:
model.fit(X_train, y_train, epochs=10)

Train on 34226 samples
Epoch 1/10
34226/34226 [==============================] - 4s 102us/sample - loss: 62.3858 - mean_absolute_error: 62.3858 - mean_squared_error: 47952.5156 - R_squared: 0.2783
Epoch 2/10
34226/34226 [==============================] - 3s 93us/sample - loss: 62.4518 - mean_absolute_error: 62.4518 - mean_squared_error: 47876.6758 - R_squared: 0.2764
Epoch 3/10
34226/34226 [==============================] - 3s 94us/sample - loss: 62.4206 - mean_absolute_error: 62.4206 - mean_squared_error: 47907.6992 - R_squared: 0.2848
Epoch 4/10
34226/34226 [==============================] - 3s 94us/sample - loss: 62.3398 - mean_absolute_error: 62.3398 - mean_squared_error: 47944.8633 - R_squared: 0.2813
Epoch 5/10
34226/34226 [==============================] - 3s 94us/sample - loss: 62.3424 - mean_absolute_error: 62.3424 - mean_squared_error: 47973.3086 - R_squared: 0.2847
Epoch 6/10
34226/34226 [==============================] - 3s 95us/sample - loss: 62.3229 - mean_absolute_error:

In [ ]:
model.evaluate(X_test, y_test)

## Conclusions